In [2]:
import pandas as pd
import numpy as np

In [3]:
trade_mx = pd.read_csv("trade_matrix_import_only.csv")

In [4]:
codes = pd.read_csv("country_to_continent_codes.csv", )

In [5]:
colstodrop = ['Unnamed: 0', 'Item Code', 'Flag']

trade_mx.drop(colstodrop, axis=1, inplace=True)

In [6]:
trade_mx.head()

,Reporter Country Code,Partner Country Code,Year,Unit,Value
0,3,3,2004,t,12.00
1,4,2,2005,t,3.00
2,3,3,2005,t,52.00
3,7,2,2022,t,25.06
4,4,2,1996,t,171.00


In [7]:
codes

,Area Code,Continent,Country,Capital,Latitude,Longitude
0,1,europe,armenia,yerevan,40.183300,44.516700
1,2,asia,afghanistan,kabul,34.526000,69.181000
2,3,europe,albania,tirana,41.327500,19.818900
3,4,africa,algeria,algiers,36.737232,3.086472
4,7,africa,angola,luanda,-8.839988,13.289437
...,...,...,...,...,...,...
217,272,europe,serbia,belgrade,44.786600,20.448900
218,273,europe,montenegro,podgorica,42.441300,19.263600
219,276,africa,sudan,khartoum,15.500654,32.559899
220,277,africa,s. sudan,juba,4.851650,31.582470


In [8]:
# Merging the DataFrames on the country codes
trade_mx = trade_mx.merge(codes[['Area Code', 'Continent']], left_on='Reporter Country Code', right_on='Area Code', how='left')
trade_mx = trade_mx.rename(columns={'Continent': 'Reporter Continent'})
trade_mx = trade_mx.drop(columns='Area Code')

In [9]:
trade_mx = trade_mx.merge(codes[['Area Code', 'Continent']], left_on='Partner Country Code', right_on='Area Code', how='left')
trade_mx = trade_mx.rename(columns={'Continent': 'Partner Continent'})
trade_mx = trade_mx.drop(columns='Area Code')

In [10]:
trade_mx.head()

,Reporter Country Code,Partner Country Code,Year,Unit,Value,Reporter Continent,Partner Continent
0,3,3,2004,t,12.00,europe,europe
1,4,2,2005,t,3.00,africa,asia
2,3,3,2005,t,52.00,europe,europe
3,7,2,2022,t,25.06,africa,asia
4,4,2,1996,t,171.00,africa,asia


In [11]:
# Selecting and renaming the necessary columns
trade_mx = trade_mx[['Reporter Continent', 'Partner Continent', 'Year', 'Unit', 'Value']]

# Grouping by year and summing the values
continent_mx = trade_mx.groupby(['Reporter Continent', 'Partner Continent', 'Year']).agg({'Value': 'sum'}).reset_index()

In [12]:
# Renaming for sankey code
continent_mx = continent_mx.rename(columns={
    'Reporter Continent': 'target',
    'Partner Continent': 'source',
    'Year': 'year',
    'Value': 'value'
})

In [13]:
continent_mx = continent_mx[['year', 'source', 'target', 'value']]

In [14]:
continent_mx.to_csv("continent_trade_matrix.csv", index=False)

In [15]:
continent_mx['target'].value_counts()

target
europe           226
africa           225
asia             225
north america    223
oceania          223
south america    222
Name: count, dtype: int64